# Agent Scenarios

### Simple Single Agent Scenario

In [ ]:
import os
from agents import Agent, Runner, trace
import asyncio

from dotenv import load_dotenv
load_dotenv()

In [ ]:
os.environ["OPENAI_BASE_URL"] = "http://localhost:11434/v1"
os.environ["OPENAI_API_KEY"] = "******"
os.environ["OPENAI_TRACING_ENABLED"] = "false"

agent = Agent(
    name="Test Agent",
    instructions="You are financial advisor",
    model="gemma3:1b",
)


result = await Runner.run(agent, input="Provide fundamental analysis of Tesla.")
print("Result:", result.final_output)

In [ ]:
with trace("Telling me a joke"):
    result = await Runner.run(agent, input="Tell me a joke.")
    print("Result:", result)
    

In [ ]:
result = Runner.run_streamed(agent, input="Provide fundamental analysis of Tesla.")
async for event in result.stream_events():
    if event.type == "raw_response_event" and event.data.type == "response.output_text.delta":
            print(event.data.delta, end="", flush=True)


### Multi-agent Scenario

In [ ]:
import os

# Force LM Studio
os.environ["OPENAI_BASE_URL"] = "http://localhost:1234/v1"
os.environ["OPENAI_API_KEY"] = "lm-studio"

from agents import Agent, Runner

agent = Agent(
    name="DTL Agent",
    instructions="Return a short numbered list of AI use cases.",
    model="openai/gpt-oss-20b",
)

# NOTE: no await here
result = Runner.run_streamed(
    agent,
    input="Identify 5 AI use cases for life insurance call center payments and bank account change requests.",
)

async for event in result.stream_events():
    if event.type == "raw_response_event" and event.data.type == "response.output_text.delta":
        print(event.data.delta, end="", flush=True)

print("\n\nFINAL:\n", result.final_output)


In [ ]:
import os
from agents import Agent, Runner  # OpenAI Agents SDK

# 1) Point to LM Studio's OpenAI-compatible server (MUST include /v1)
# os.environ["OPENAI_BASE_URL"] = os.getenv("LMSTUDIO_API_BASE_URL", "http://localhost:1234/v1")
# os.environ["OPENAI_API_KEY"] = os.getenv("LMSTUDIO_API_KEY", "lm-studio")

os.environ["OPENAI_BASE_URL"] =  "http://localhost:1234/v1"
os.environ["OPENAI_API_KEY"] =  "lm-studio"



agent = Agent(
    name="DTL Agent",
    instructions="Return a short numbered list of AI use cases.",
    model="openai/gpt-oss-20b",
)

# Jupyter: use `await` (no asyncio.run)
result = Runner.run_streamed(
    agent,
    input="Identify 5 AI use cases for life insurance call center payments and bank account change requests.",
)

async for event in result.stream_events():
    if event.type == "raw_response_event" and event.data.type == "response.output_text.delta":
        print(event.data.delta, end="", flush=True)

print("\n\nFINAL:\n", result.final_output)


In [ ]:
AgentInstruction1 = "You are Digital Transformation Leader with AI expertise who identifies the AI use cases in different industries."
AgentInstruction2 = "You are AI expert assessing AI technologies for business impact with ROI. who can identify high impact one best AI use cases which is very feasible to implement and impact is very high."
AgentInstruction3 = "You are AI expert who creates AI product requirement documents."
AgentInstruction4 = "You are are pessimistic AI expert who analyzes AI risks and challenges and finds faults in any automation ideas."

In [ ]:
agent1 = Agent(name ="DTL Agent", instructions=AgentInstruction1, model="openai/gpt-oss-20b")
result1 = Runner.run_streamed(agent1, input="Identify AI use cases in Life Insurance Call center industry: the customer support operations attends the call and if the customer asks to change the bank account number or to make a payment then he validates the customer credentials and checks the policy details for payment due and applies the ACH payments including bank account number change requests based on the information captured along with the payment schedule. Limit to only 1 use case")
async for event in result1.stream_events():
    if event.type == "raw_response_event" and event.data.type == "response.output_text.delta":
            print(event.data.delta, end="", flush=True)
print(result1.final_output)
usecases = result1.final_output

In [ ]:
print(agent1)

In [ ]:
agent2 = Agent(name="AI Business Impact Assessor",    instructions=AgentInstruction2,    model="openai/gpt-oss-20b",)
result2 = Runner.run_streamed(agent2, input=f"Assess AI technologies for business impact and ROI in {usecases}")
async for event in result2.stream_events():
    if event.type == "raw_response_event" and event.data.type == "response.output_text.delta":
            print(event.data.delta, end="", flush=True)
print(result2.final_output)
usecase_analysis = result2.final_output

In [ ]:
agent3 = Agent(    name="AI Product Requirement Document Creator",    instructions=AgentInstruction3,    model="openai/gpt-oss-20b",)
result3 = Runner.run_streamed(agent3, input=f"Create AI product requirement document for {usecases} based on the following analysis {usecase_analysis}")
async for event in result3.stream_events():
    if event.type == "raw_response_event" and event.data.type == "response.output_text.delta":
            print(event.data.delta, end="", flush=True)
print(result3.final_output)
prd = result3.final_output

In [ ]:
agent4 = Agent(    name="Pessimistic AI Expert",    instructions=AgentInstruction4,    model="openai/gpt-oss-20b",)
result4 = Runner.run_streamed(agent4, input=f"Analyze risks and challenges in automating the following use case analysis: {usecase_analysis}")
async for event in result4.stream_events():
    if event.type == "raw_response_event" and event.data.type == "response.output_text.delta":
            print(event.data.delta, end="", flush=True)
print(result4.final_output)
risks = result4.final_output

In [ ]:
from agents import function_tool

In [ ]:
body = f"""Hello, This is automated email containing the AI use case analysis report."
\n\nIdentified Use Cases:\n{usecases}\n\nUse Case Analysis:\n{usecase_analysis}\n\nProduct Requirement Document:\n{prd}\n\n\n\nRegards,
AI Research Bot"""

@function_tool
def send_email(body: str):
    host = os.getenv("debugmailhost")
    port = int(os.getenv("debugmailport"))
    login = os.getenv("debugmaillogin")
    password = os.getenv("debugmailpassword")
    from email.mime.text import MIMEText
    from email.mime.multipart import MIMEMultipart
    import smtplib 
    msg = MIMEMultipart()
    msg['From'] = "blitznihar@gmail.com"
    msg['To'] = "recipient@example.com"
    msg['Subject'] = "AI Use Case Analysis Report"
    msg.attach(MIMEText(body, 'html'))
    server = smtplib.SMTP(host, port)
    server.starttls()
    server.login(login, password)
    text = msg.as_string()
    server.sendmail(msg['From'], msg['To'], text)
    server.quit()
    return {"status": "success"}
    
# send_email(body)

In [ ]:
tool1 = agent1.as_tool(tool_name="Identify AI Use Cases", tool_description="Identifies AI use cases in a given industry scenario.")
tool2 = agent2.as_tool(tool_name="Assess AI Business Impact", tool_description="Assesses AI technologies for business impact and ROI.")
tool3 = agent3.as_tool(tool_name="Create AI Product Requirement Document", tool_description="Creates AI product requirement documents based on use case analysis.")
tool4 = agent4.as_tool(tool_name="Analyze AI Risks", tool_description="Analyzes risks and challenges in automating AI use cases.")


In [ ]:
FinalAgentInstruction = """You are AI Research Bot who uses following tools to identify AI use cases, assess their business impact, create product requirement documents and analyze risks.
Use the tools to perform the tasks step by step and finally send an email with the complete analysis report. you send test email to: blitznihar@gmail.com from recipient@example.co with final report of the analysis as the body.
Tools:
1. Identify AI Use Cases
2. Assess AI Business Impact
3. Create AI Product Requirement Document
4. Send Email
"""
final_agent = Agent(    name="AI Research Bot",
    instructions=FinalAgentInstruction,
    model="openai/gpt-oss-20b",
    tools=[tool1, tool2, tool3, tool4, send_email],
)
final_result = await Runner.run(final_agent, 
                                input="Perform complete AI use case analysis and send email report in html format for Life Insurance Call center industry: the customer support operations attends the call and if the customer asks to change the bank account number or to make a payment then he validates the customer credentials and checks the policy details for payment due and applies the ACH payments including bank account number change requests based on the information captured along with the payment schedule")
print("Final Result:", final_result.final_output)
finalreport = final_result.final_output


In [ ]:
AgentInstruction5 = "you create the final email body with the complete analysis report"
agent5 = Agent(    name="Final Email Body Creator",    instructions=AgentInstruction5,    model="openai/gpt-oss-20b",)
result5 = Runner.run_streamed(agent5, input=f"Create the final email body with the complete analysis report including identified use cases, use case analysis, product requirement document and risks and challenges. {finalreport}")
# async for event in result5.stream_events():
#     if event.type == "raw_response_event" and event.data.type == "response.output_text.delta":
#             print(event.data.delta, end="", flush=True)

tool5 = agent5.as_tool(tool_name="Final Email Body Creator", tool_description="Creates the final email body with the complete analysis report: {finalreport}.")      


FinalAgentInstruction = "you send final email to: blitznihar@gmail.com from recipient@example.com with body: final report: {finalreport} as body "
final_agent = Agent(name="AI Research Bot",
    instructions=FinalAgentInstruction,
    model="openai/gpt-oss-20b",
    tools=[tool5, send_email]
)

final_result = await Runner.run(final_agent, 
                                input=f"Create the final email body in html format with the complete analysis report including identified use cases, use case analysis, product requirement document and risks and challenges. {finalreport} and send email to blitznihar@gmail.com from recipient@example.com")
print("Final Result:", final_result.final_output)